<h1 style="text-align: center;"> Model Tuning </p>

<a id='Table-of-Contents'></a>


##  Table of Contents
1. [Required Libraries](#Required-Libraries)
2. [Load Data](#Load-Data)
3. 

## Required Libraries

[[ go back to the top ]](#Table-of-contents)

This notebook uses several Python libraries such as:

In [12]:
# Load required packages 
import datetime
from datetime import timedelta
import numpy as np
import pandas as pd

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Time-Series
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARMA, ARIMA
from scipy import signal
import scipy.stats as stats

from Functions import LOAD_DATA

import warnings
warnings.filterwarnings("ignore")

<a id='Load-Data'></a>

---
## Load Data

[[ go back to the top ]](#Table-of-Contents)

In [13]:
filepath = './'
filename = 'model_ready_data.csv'
LOAD_DATA(filepath, filename)

NameError: name 'pd' is not defined

In [14]:
# Load Data Function
def LOAD_DATA(filepath, filename):
    # Read CSV files
    if filename.endswith('.csv'):
        new_df = pd.read_csv(filepath+filename)

    # Read Excel files
    elif filename.endswith('.xlsx'):
        new_df = pd.read_excel(filepath+filename)
    print(type(new_df.index))
    if type(new_df.index) != pd.core.indexes.datetimes.DatetimeIndex:
        for col in new_df.columns:
            if col.lower().find('date') != -1:
                print(f"TIMESTAMP FOUND! '{col}'")
                print()
                new_df['date'] = pd.to_datetime(new_df[col]) # format = '%Y/%m/%d'
                new_df.set_index('date', inplace = True)
                # If datetime col was already == 'date', no need to drop col after set_index, otherwise...
                if col != 'date':
                    new_df.drop(columns = col, inplace = True)
                
    # Try to identify the date column
    elif type(new_df.index) == pd.core.indexes.datetimes.DatetimeIndex:
        print('Index already in datetime')
        
    display(new_df.info())
    return new_df

In [16]:
filepath = './'
filename = 'model_ready_data.csv'
LOAD_DATA(filepath, filename)

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'date'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 244 entries, 2020-02-18 to 2021-02-25
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   phillips_66                        244 non-null    float64
 1   bp_plc                             244 non-null    float64
 2   valero_energy_corporation          244 non-null    float64
 3   chevron_corporation                244 non-null    float64
 4   occidental_petroleum_corporation   244 non-null    float64
 5   marathon_oil_corporation           244 non-null    float64
 6   pioneer_natural_resources_company  244 non-null    float64
 7   conocophillips                     244 non-null    float64
 8   exxon_mobil_corporation            244 non-null    float64
 9   marathon_petroleum_corporation     244 non-null    float64
 10  workplaces                     

None

,phillips_66,bp_plc,valero_energy_corporation,chevron_corporation,occidental_petroleum_corporation,marathon_oil_corporation,pioneer_natural_resources_company,conocophillips,exxon_mobil_corporation,marathon_petroleum_corporation,workplaces,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,int_news_sentiment,na_news_sentiment
date,,,,,,,,,,,,,,,,,,
2020-02-18,88.00,36.09,82.52,110.24,41.56,10.30,135.56,58.30,59.88,56.47,-0.006305,-0.211659,-1.377153,1.345745,2.440042,5.466077,0.00,0.00
2020-02-19,88.36,36.25,83.23,110.74,42.51,10.47,141.17,59.26,60.34,57.69,1.001656,2.222899,0.639556,0.345455,2.634615,8.153166,0.00,0.00
2020-02-20,90.19,35.98,84.17,109.81,42.97,10.25,142.53,58.88,59.86,60.26,-0.387448,0.192474,-0.167570,1.922258,2.098260,-1.300847,-0.20,0.00
2020-02-21,89.25,35.36,82.90,109.01,42.12,10.11,142.25,58.44,59.13,59.13,0.547162,2.786383,-1.632662,1.082046,3.656008,6.725381,0.00,0.00
2020-02-24,86.56,34.06,78.10,104.71,39.48,9.46,135.97,56.38,56.36,55.87,3.010224,1.766222,0.129503,0.641213,0.961066,7.336245,0.10,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-19,82.31,22.87,71.53,95.80,25.42,9.45,133.94,48.42,52.37,53.00,-26.703297,-20.935180,-12.503418,12.225713,-25.787372,-26.599754,0.00,0.00
2021-02-22,83.96,23.63,74.25,98.39,26.47,10.20,139.47,50.88,54.30,54.85,-19.689617,-14.583490,-8.355705,8.119504,-18.301961,-17.634278,0.00,0.00
2021-02-23,85.53,24.23,75.81,99.63,26.06,11.16,145.24,52.10,55.05,55.77,-19.324188,-8.923650,-3.151351,6.976641,-14.679208,2.265554,-0.27,-0.50
